In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import lifelines
import pycox
import xgboost as xgb
from sklearn.model_selection import train_test_split

from xgbse import XGBSEKaplanNeighbors, XGBSEKaplanTree, XGBSEDebiasedBCE, XGBSEBootstrapEstimator, XGBSEStackedWeibull
from xgbse.converters import convert_data_to_xgb_format, convert_to_structured, convert_y
from xgbse.non_parametric import get_time_bins

from benchmark import BenchmarkLifelines, BenchmarkXGBoost, BenchmarkXGBSE, BenchmarkPysurvival
#from pysurvival.models.survival_forest import ConditionalSurvivalForestModel

# setting seed
np.random.seed(42)

In [2]:
def split_train_test_valid(dataf, test_size=.2, valid_size=.01, random_state=1):
    df_train, df_test = train_test_split(dataf, test_size=test_size, random_state=random_state)
    df_train, df_valid = train_test_split(df_train, test_size=valid_size, random_state=random_state)
    return df_train, df_valid,  df_test

## Data

In [3]:
from pycox.datasets import flchain
df = flchain.read_df()
df = df.rename(columns={'death':'event', 'futime':'duration'})
df = df.astype(float)

df_train, df_valid, df_test = split_train_test_valid(df)

T_train = df_train.duration
E_train = df_train.event
TIME_BINS = time_bins = get_time_bins(T_train, E_train, size=30)

Dataset 'flchain' not locally available. Downloading...
Done


## Model

Let us fit a model and check performance.

In [4]:
results = pd.DataFrame()

## Lifelines

In [5]:
bench_lifelines = BenchmarkLifelines(lifelines.CoxPHFitter(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Cox-PH"
)

In [6]:
bench_lifelines.train()

In [7]:
cox_lifelines_results = bench_lifelines.test()
cox_lifelines_results

{'model': 'Cox-PH',
 'c-index': 0.7883798521392772,
 'ibs': 0.0985487804655452,
 'dcal_pval': 0.9711460792177572,
 'dcal_max_dev': 0.0113195501393272,
 'training_time': 0.9423878192901611,
 'inference_time': 0.004564046859741211}

In [8]:
results = results.append(cox_lifelines_results, ignore_index=True)

In [9]:
bench_lifelines = BenchmarkLifelines(lifelines.WeibullAFTFitter(),
        df_train.assign(duration = np.where(df_train.duration == 0, 1, df_train.duration)),
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Weibull AFT"
)

In [10]:
bench_lifelines.train()
weibull_aft_results = bench_lifelines.test()
weibull_aft_results

{'model': 'Weibull AFT',
 'c-index': 0.788764446796106,
 'ibs': 0.09899526138319895,
 'dcal_pval': 0.8485679576993114,
 'dcal_max_dev': 0.013089751262202395,
 'training_time': 0.5366322994232178,
 'inference_time': 0.005728960037231445}

In [11]:
results = results.append(weibull_aft_results, ignore_index=True)

## XGBoost

In [12]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - AFT",
        "survival:aft",
)

In [13]:
bench_xgboost.train()
xgboost_aft_results = bench_xgboost.test()
xgboost_aft_results

{'model': 'XGB - AFT',
 'c-index': 0.771647534919725,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.1057589054107666,
 'inference_time': 0.0008881092071533203}

In [14]:
results = results.append(xgboost_aft_results, ignore_index=True)

In [15]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - Cox",
        "survival:cox",
)
bench_xgboost.train()
xgboost_cox_results = bench_xgboost.test()
xgboost_cox_results

{'model': 'XGB - Cox',
 'c-index': 0.7748247277216809,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 0.05382394790649414,
 'inference_time': 0.0006210803985595703}

In [16]:
results = results.append(xgboost_cox_results, ignore_index=True)

## XGB SE

In [17]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEDebiasedBCE(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Debiased BCE",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bce_results = bench_xgboost_embedding.test()
xgboost_bce_results

{'model': 'XGBSE - Debiased BCE',
 'c-index': 0.7843379337713303,
 'ibs': 0.11658577957742053,
 'dcal_pval': 6.499812880315087e-05,
 'dcal_max_dev': 0.03715199563137006,
 'training_time': 3.061776876449585,
 'inference_time': 0.23334813117980957}

In [18]:
results = results.append(xgboost_bce_results, ignore_index=True)

In [19]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanNeighbors(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Neighbors",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kn_results = bench_xgboost_embedding.test()
xgboost_kn_results

{'model': 'XGBSE - Kaplan Neighbors',
 'c-index': 0.7765872490948552,
 'ibs': 0.10166555304551815,
 'dcal_pval': 0.9177066941432961,
 'dcal_max_dev': 0.012590080176862473,
 'training_time': 0.47940921783447266,
 'inference_time': 0.542578935623169}

In [20]:
results = results.append(xgboost_kn_results, ignore_index=True)

In [21]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanTree(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Tree",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kt_results = bench_xgboost_embedding.test()
xgboost_kt_results

{'model': 'XGBSE - Kaplan Tree',
 'c-index': 0.767830984121385,
 'ibs': 0.10346911400864475,
 'dcal_pval': 0.9287914257772109,
 'dcal_max_dev': 0.01106687028007211,
 'training_time': 0.16666603088378906,
 'inference_time': 0.003175973892211914}

In [22]:
results = results.append(xgboost_kt_results, ignore_index=True)

In [23]:
base_tree = XGBSEKaplanTree()

bench_xgboost_embedding = BenchmarkXGBSE(
    XGBSEBootstrapEstimator(base_tree, n_estimators=100),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Bootstrap Trees",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bootstrap_results = bench_xgboost_embedding.test()
xgboost_bootstrap_results

{'model': 'XGBSE - Bootstrap Trees',
 'c-index': 0.7814485745501223,
 'ibs': 0.10047335035894789,
 'dcal_pval': 0.9845121854944557,
 'dcal_max_dev': 0.009485243521309514,
 'training_time': 15.351151943206787,
 'inference_time': 0.382260799407959}

In [24]:
results = results.append(xgboost_bootstrap_results, ignore_index=True)

In [25]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEStackedWeibull(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Stacked Weibull",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_weibull_results = bench_xgboost_embedding.test()
xgboost_weibull_results

{'model': 'XGBSE - Stacked Weibull',
 'c-index': 0.7763582070540049,
 'ibs': 0.10295151190387516,
 'dcal_pval': 0.993686931213167,
 'dcal_max_dev': 0.007788526213200461,
 'training_time': 0.7186770439147949,
 'inference_time': 0.011139869689941406}

In [26]:
results = results.append(xgboost_weibull_results, ignore_index=True)

In [27]:
results.sort_values("c-index", ascending=False).round(3)

,c-index,dcal_max_dev,dcal_pval,ibs,inference_time,model,training_time
1,0.789,0.013,0.849,0.099,0.006,Weibull AFT,0.537
0,0.788,0.011,0.971,0.099,0.005,Cox-PH,0.942
4,0.784,0.037,0.000,0.117,0.233,XGBSE - Debiased BCE,3.062
7,0.781,0.009,0.985,0.100,0.382,XGBSE - Bootstrap Trees,15.351
5,0.777,0.013,0.918,0.102,0.543,XGBSE - Kaplan Neighbors,0.479
8,0.776,0.008,0.994,0.103,0.011,XGBSE - Stacked Weibull,0.719
3,0.775,NaN,NaN,NaN,0.001,XGB - Cox,0.054
2,0.772,NaN,NaN,NaN,0.001,XGB - AFT,0.106
6,0.768,0.011,0.929,0.103,0.003,XGBSE - Kaplan Tree,0.167


In [28]:
import tabulate
results.sort_values("c-index", ascending=False).round(3).set_index('model').to_markdown()

'| model                    |   c-index |   dcal_max_dev |   dcal_pval |     ibs |   inference_time |   training_time |\n|:-------------------------|----------:|---------------:|------------:|--------:|-----------------:|----------------:|\n| Weibull AFT              |     0.789 |          0.013 |       0.849 |   0.099 |            0.006 |           0.537 |\n| Cox-PH                   |     0.788 |          0.011 |       0.971 |   0.099 |            0.005 |           0.942 |\n| XGBSE - Debiased BCE     |     0.784 |          0.037 |       0     |   0.117 |            0.233 |           3.062 |\n| XGBSE - Bootstrap Trees  |     0.781 |          0.009 |       0.985 |   0.1   |            0.382 |          15.351 |\n| XGBSE - Kaplan Neighbors |     0.777 |          0.013 |       0.918 |   0.102 |            0.543 |           0.479 |\n| XGBSE - Stacked Weibull  |     0.776 |          0.008 |       0.994 |   0.103 |            0.011 |           0.719 |\n| XGB - Cox                |     0.775 